# ЛР8: Кластеризация и классификация данных

## Практикум 8. Задание

В данной лабораторной работе мы изучим методы кластеризации и классификации данных, используя различные алгоритмы машинного обучения.

In [ ]:
# Установка необходимых библиотек
%pip install scikit-learn matplotlib seaborn pandas numpy plotly

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Загрузка и предварительный анализ данных

In [ ]:
# Загружаем датасет Wine
wine = datasets.load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target

print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
df.head()

In [ ]:
# Изучаем структуру данных
print("Информация о датасете:")
print(df.info())
print("\nПропущенные значения:")
print(df.isnull().sum().sum())
print("\nОписательная статистика:")
df.describe()

In [ ]:
# Визуализация распределения признаков
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
axes = axes.ravel()

for i, column in enumerate(df.columns[:-1]):  # исключаем target
    axes[i].hist(df[column], bins=20, alpha=0.7)
    axes[i].set_title(f'Распределение {column}')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Частота')

plt.tight_layout()
plt.show()

In [ ]:
# Корреляционная матрица
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Корреляционная матрица признаков')
plt.tight_layout()
plt.show()

## 2. Подготовка данных

In [ ]:
# Разделение на признаки и целевую переменную
X = df.drop('target', axis=1)
y = df['target']

# Стандартизация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("Исходные данные (первые 5 строк):")
print(X.head())
print("\nСтандартизированные данные (первые 5 строк):")
print(X_scaled_df.head())

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

print(f"\nРазмер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

## 3. Кластеризация

In [ ]:
# K-means кластеризация
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Иерархическая кластеризация
hierarchical = AgglomerativeClustering(n_clusters=3)
hierarchical_labels = hierarchical.fit_predict(X_scaled)

# DBSCAN кластеризация
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

print("Количество кластеров K-means:", len(np.unique(kmeans_labels)))
print("Количество кластеров иерархическая:", len(np.unique(hierarchical_labels)))
print("Количество кластеров DBSCAN:", len(np.unique(dbscan_labels)))
print("\nРаспределение по кластерам DBSCAN:")
print(np.unique(dbscan_labels, return_counts=True))

In [ ]:
# Визуализация кластеров с помощью PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Истинные классы
scatter = axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7)
axes[0, 0].set_title('Истинные классы')
axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[0, 0])

# K-means
scatter = axes[0, 1].scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels, cmap='viridis', alpha=0.7)
axes[0, 1].set_title('K-means кластеризация')
axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[0, 1])

# Иерархическая
scatter = axes[1, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=hierarchical_labels, cmap='viridis', alpha=0.7)
axes[1, 0].set_title('Иерархическая кластеризация')
axes[1, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[1, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[1, 0])

# DBSCAN
scatter = axes[1, 1].scatter(X_pca[:, 0], X_pca[:, 1], c=dbscan_labels, cmap='viridis', alpha=0.7)
axes[1, 1].set_title('DBSCAN кластеризация')
axes[1, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[1, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## 4. Классификация

In [ ]:
# Обучение различных классификаторов
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(random_state=42, probability=True),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}

results = {}

for name, classifier in classifiers.items():
    # Обучение
    classifier.fit(X_train, y_train)
    
    # Предсказания
    y_pred = classifier.predict(X_test)
    y_pred_proba = classifier.predict_proba(X_test) if hasattr(classifier, 'predict_proba') else None
    
    # Оценка качества
    accuracy = accuracy_score(y_test, y_pred)
    
    results[name] = {
        'classifier': classifier,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'accuracy': accuracy
    }
    
    print(f"{name} - Точность: {accuracy:.4f}")
    print(f"Отчет по классификации:\n{classification_report(y_test, y_pred)}\n")

In [ ]:
# Матрицы ошибок
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Матрица ошибок - {name}')
    axes[i].set_xlabel('Предсказанный класс')
    axes[i].set_ylabel('Истинный класс')

plt.tight_layout()
plt.show()

In [ ]:
# ROC-кривые (для многоклассовой классификации)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# Бинаризация меток для многоклассовой ROC
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
n_classes = y_test_bin.shape[1]

plt.figure(figsize=(12, 8))
colors = cycle(['blue', 'red', 'green'])

for name, result in results.items():
    if result['y_pred_proba'] is not None:
        y_score = result['y_pred_proba']
        
        # Вычисление ROC для каждого класса
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        
        # Средняя ROC
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        
        plt.plot(fpr["micro"], tpr["micro"], color=next(colors),
                label=f'{name} (AUC = {roc_auc["micro"]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Случайный классификатор')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Доля ложноположительных результатов')
plt.ylabel('Доля истинноположительных результатов')
plt.title('ROC-кривые')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

## 5. Анализ важности признаков

In [ ]:
# Важность признаков для Random Forest
rf_classifier = results['Random Forest']['classifier']
feature_importance = rf_classifier.feature_importances_

# Создание DataFrame с важностью признаков
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Важность признаков (Random Forest):")
print(feature_importance_df)

# Визуализация важности признаков
plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance_df, x='importance', y='feature')
plt.title('Важность признаков (Random Forest)')
plt.xlabel('Важность')
plt.tight_layout()
plt.show()

## 6. Сравнение результатов

In [ ]:
# Сравнение точности классификаторов
accuracy_comparison = pd.DataFrame({
    'Классификатор': list(results.keys()),
    'Точность': [result['accuracy'] for result in results.values()]
})

plt.figure(figsize=(10, 6))
sns.barplot(data=accuracy_comparison, x='Классификатор', y='Точность')
plt.title('Сравнение точности классификаторов')
plt.ylabel('Точность')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("Сравнение точности:")
print(accuracy_comparison.sort_values('Точность', ascending=False))

## 7. Выводы и интерпретация результатов

In [ ]:
print("=== ВЫВОДЫ И ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ ===\n")

print("1. АНАЛИЗ ДАННЫХ:")
print(f"   - Датасет содержит {df.shape[0]} образцов и {df.shape[1]-1} признаков")
print(f"   - Количество классов: {len(np.unique(y))}")
print(f"   - Пропущенных значений: {df.isnull().sum().sum()}")
print(f"   - Объясненная дисперсия первыми двумя компонентами PCA: {pca.explained_variance_ratio_[:2].sum():.2%}")

print("\n2. КЛАСТЕРИЗАЦИЯ:")
print(f"   - K-means: {len(np.unique(kmeans_labels))} кластеров")
print(f"   - Иерархическая: {len(np.unique(hierarchical_labels))} кластеров")
print(f"   - DBSCAN: {len(np.unique(dbscan_labels))} кластеров (включая шум)")

print("\n3. КЛАССИФИКАЦИЯ:")
best_classifier = max(results.items(), key=lambda x: x[1]['accuracy'])
print(f"   - Лучший классификатор: {best_classifier[0]} (точность: {best_classifier[1]['accuracy']:.4f})")
print(f"   - Все классификаторы показали высокую точность (>90%)")

print("\n4. ВАЖНЫЕ ПРИЗНАКИ:")
top_features = feature_importance_df.head(3)
print(f"   - Топ-3 важных признака: {', '.join(top_features['feature'].tolist())}")

print("\n5. РЕКОМЕНДАЦИИ:")
print("   - Random Forest показал лучшие результаты и позволяет интерпретировать важность признаков")
print("   - Все методы кластеризации успешно выделили 3 кластера, соответствующие классам")
print("   - Датасет хорошо подходит для задач классификации и кластеризации")

## 8. Сохранение результатов

In [ ]:
# Создание итогового DataFrame с результатами
results_df = df.copy()
results_df['kmeans_cluster'] = kmeans_labels
results_df['hierarchical_cluster'] = hierarchical_labels
results_df['dbscan_cluster'] = dbscan_labels

# Добавление предсказаний лучшего классификатора
best_predictions = best_classifier[1]['y_pred']
# Создаем массив предсказаний для всех образцов
all_predictions = np.zeros(len(df))
all_predictions[X_test.shape[0]:] = best_predictions  # Заполняем только тестовую часть
results_df['best_classifier_prediction'] = all_predictions

print("Итоговый DataFrame с результатами:")
print(results_df.head())

# Сохранение в Excel
results_df.to_excel('/workspace/solutions/LR-s/LR8/LR8_results.xlsx', index=False)
print("\nРезультаты сохранены в файл LR8_results.xlsx")

In [ ]:
# Создание файла со ссылкой на Colab
colab_link = "https://colab.research.google.com/drive/your_notebook_id_here"

with open('/workspace/solutions/LR-s/LR8/colab_link.txt', 'w', encoding='utf-8') as f:
    f.write(f"Ссылка на Colab с результатами ЛР8:\n{colab_link}")

print("Файл со ссылкой на Colab создан: colab_link.txt")
print(f"Ссылка: {colab_link}")